# 2D -> 3D Data Processing
Execution of this notebook is meant to follow `1_data_processing.ipynb`. 

The 

In [2]:
import os
from __init__ import data_path

In [5]:
from mmpose.apis import MMPoseInferencer
inferencer = MMPoseInferencer('human')

ModuleNotFoundError: No module named 'mmpose'

In [4]:
dataset = "tenniset"

In [ ]:
from torch import Dataset # Fix

class VideoAndKeypointDataset(Dataset):
    def __init__(self,
                 dataset_path: str):
        self.dataset_rooot_path = dataset_path

        self.segments_path = os.path.join(dataset_path, "segments")
        self.labels_path = os.path.join(dataset_path, "labels")

        self.segments = os.listdir(self.segments_path)
        self.labels = os.listdir(self.labels_path)

    def __len__(self):
        """
        We are measuring length in terms of the number of video segments (~ number of points played)
        """
        return len(self.segments)

    def __getitem__(self, idx):
        poses_2d = self._get_player_poses(idx)
        cropped_video = self._get_segment_video(idx)

        assert len(poses_2d) == len(cropped_video)

        sample = {
            poses_2d: poses_2d,
            cropped_video: cropped_video
        }

        return sample

    def _get_segment_video(self, idx):
        video_file_name = self.segments[idx]
        video_file = os.path.join(self.segments_path, video_file_name)

        video_frames = [] # TODO: load video

        return video_frames
    
    def _get_segment_poses(self, idx):
        label_file_name = self.labels[idx]
        player_pose_file = os.path.join(self.labels_path, label_file_name)
        poses = [] # TODO: load file contents

        return poses
